In [1]:
!pip install scikit-surprise
#wanted a download as this is a popular tool for recommender systems and I will be using precision@k
#and recall@k for my metrics

In [2]:
from collections import defaultdict
import pandas as pd
import numpy as np
import scipy
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import surprise as sp
import time

I need to bring back some of the code from EDA as there are some specific variables there that will need to be used for the Preprocessing and Model element.

In [3]:
user_df = pd.read_csv('cleaned_user_list.csv')
anime_df = pd.read_csv('cleaned_anime_list.csv')

In [4]:
user_anime_df = pd.read_csv('cleaned_user_anime_list.csv')

In [5]:
ratings_df = user_anime_df[['username','anime_id', 'my_score','my_status']]
ratings_df['my_score'].describe().apply(lambda x: format(x, '.2f')).reset_index()
lower_rating = ratings_df['my_score'].min()
upper_rating = ratings_df['my_score'].max()
print(f'The ratings are between: {lower_rating} to {upper_rating}')
user_ratings = ratings_df.groupby('username').size().reset_index(name='animes_rated')

The ratings are between: 0 to 10


In [6]:
sample_df_users = user_df.sample(frac = .20, random_state = 2)

user_ratings_sampled = pd.merge(
    user_ratings, sample_df_users, left_on = 'username', right_on = 'username', how = 'inner')

user_ratings_aggregated = user_ratings_sampled.groupby('animes_rated').size().reset_index(
    name='group_size').sort_values(by='animes_rated')

In [7]:
ratings_anime = ratings_df.groupby('anime_id').size().reset_index(name='number_of_users')

anime_ratings_aggregated = ratings_anime.groupby('number_of_users').size().reset_index(
    name='size_of_users').sort_values(by='number_of_users')

I need to create a method for my metrics before beginning the preprocessing and model, I am going with the Precision@k and Recall@k metric as this is a very common metric for recommender systems and it gives an importance to recommendations.

In [8]:
'''def precision_recall(predictions, k=10, threshold=7):
    
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    user_metrics = []

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])

        precision = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        recall = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

        user_metrics.append({'uid': uid, 'precision': precision, 'recall': recall})

    user_metrics = pd.DataFrame(user_metrics)

    return user_metrics
'''

def precision_recall_at_k(predictions, k=10, threshold= 7):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    # Predictions: Traz uma lista de 5 campos dentro de uma tupla com as seguintes infos: User_ID, Item_ID, True_ID, Est_ID, Details
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
    # Creates a dict with the key being a user and the value bringing the estimated rating and the true rating.

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k /  n_rel if n_rel != 0 else 1

    return precisions, recalls

We might run into this issue called a "cold start" which essentially is a problem for users that do not have historical data for us to go off of and recommend.

In [9]:
user_ratings_10 = user_ratings_sampled[user_ratings_sampled['animes_rated'] >= 10]
anime_ratings_10 = ratings_anime[ratings_anime['number_of_users'] >= 10]

In [10]:
rating_start = pd.merge(pd.merge(ratings_df, user_ratings_10, on='username', how='inner'), 
                            anime_ratings_10, on='anime_id', how='inner')

In [11]:
print('The initial dataframe has {0} registers and the sampled one has {1} rows.'.format(
    ratings_df['username'].count(), rating_start['username'].count()))

The initial dataframe has 80075851 registers and the sampled one has 16051272 rows.


In [12]:
rating_start.head()

,username,anime_id,my_score,my_status,animes_rated,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes,number_of_users
0,karthiga,21,9,1,53,2255153,3,49,1,0,0,55.31,2013-03-03,2014-02-04 01:32:00,7.43,0.0,3391.0,106378
1,Slimak,21,10,1,390,61677,79,224,0,3,84,126.17,2008-05-18,1900-05-01 05:04:00,7.77,2.0,7472.0,106378
2,MistButterfly,21,0,1,4472,2485327,66,3923,115,0,368,614.96,2013-04-25,2018-05-17 13:31:00,5.49,0.0,39893.0,106378
3,cfoordddd,21,10,1,716,61291,5,392,31,0,288,151.61,2008-05-17,1900-04-18 00:58:00,7.94,6.0,10027.0,106378
4,xTheFallenx,21,7,1,894,340873,22,655,2,3,212,131.89,2010-06-13,1900-05-16 03:20:00,8.02,0.0,8508.0,106378


In [13]:
#Ratings_start still has too much, we will need to sample this even further. I'm going reduce this to around 500,000 rows
random_state = 42
sample_fraction = 0.04
sample_df = rating_start.sample(frac=sample_fraction, random_state=random_state)


In [14]:
print('The initial dataframe has {0} registers and the sampled one has {1} rows.'.format(
    ratings_df['username'].count(), sample_df['username'].count()))

The initial dataframe has 80075851 registers and the sampled one has 642051 rows.


# Training and PreProcessing

In [15]:
reader = sp.Reader(rating_scale=(0, 10))
data = sp.Dataset.load_from_df(sample_df[['username', 'anime_id', 'my_score']], reader)
trainset, testset = sp.model_selection.train_test_split(data, test_size=.25, random_state = random_state)
analysis = defaultdict(list)

In [16]:
'''reader = sp.Reader(rating_scale=(0, 10))
data = sp.Dataset.load_from_df(sample_df[['username', 'anime_id', 'my_score']], reader)
trainset, testset = sp.model_selection.train_test_split(data, test_size=.25, random_state=random_state)
analysis = pd.DataFrame()'''

"reader = sp.Reader(rating_scale=(0, 10))\ndata = sp.Dataset.load_from_df(sample_df[['username', 'anime_id', 'my_score']], reader)\ntrainset, testset = sp.model_selection.train_test_split(data, test_size=.25, random_state=random_state)\nanalysis = pd.DataFrame()"

# Model

In [17]:
test_dict = {'SVD' : sp.SVD(random_state=random_state), 
             'SlopeOne' : sp.SlopeOne(), 
             'NMF' : sp.NMF(random_state=random_state),
             'BaselineOnly' : sp.BaselineOnly(),
             'NormalPredictor' : sp.NormalPredictor(), 
             }

In [18]:
try:
    for key, value in test_dict.items():
        print(f"Training {value}")
        start = time.time()    
        value.fit(trainset)
        predictions = value.test(testset)

        rmse = sp.accuracy.rmse(predictions)
        precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=7)
        precision_avg = sum(prec for prec in precisions.values()) / len(precisions)

        analysis[value] = (key, rmse, precision_avg, time.time() - start)

    print(analysis)
except Exception as e:
    print(f"Error while training {value}")
    print(e)


Training <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7faed70de950>
RMSE: 3.5072
Training <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x7faed70dc370>
RMSE: 3.5296
Training <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x7faed70dc0a0>
RMSE: 3.7779
Training <surprise.prediction_algorithms.baseline_only.BaselineOnly object at 0x7faed70dc0d0>
Estimating biases using als...
RMSE: 3.3672
Training <surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x7faed70dcca0>
RMSE: 5.0783
defaultdict(<class 'list'>, {<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7faed70de950>: ('SVD', 3.50718738687803, 0.9485712686288662, 10.297825813293457), <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x7faed70dc370>: ('SlopeOne', 3.529564882302046, 0.8959691907182773, 13.145518064498901), <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x7faed70dc0a0>: ('NMF', 3.7778662718659652, 0.

I initially had KNN models, however it was causing my kernel to die, and lose memory. I made the decision to not use any of the KNN models, and will only be analyzing the ones above.

In [19]:
analysis_df = pd.DataFrame.from_dict(analysis, orient = 'index', 
                                     columns = ['Algorithm', 'RMSE', 'Precision@10', 'Time to run (in seconds)']).reset_index()

In [20]:
analysis_df = analysis_df[['Algorithm', 'RMSE', 'Precision@10', 'Time to run (in seconds)']]
analysis_df = analysis_df.sort_values(by=['Precision@10'], ascending = False)
analysis_df['RMSE^-1'] = analysis_df['RMSE'] ** -1
analysis_df.head(n = 15)

,Algorithm,RMSE,Precision@10,Time to run (in seconds),RMSE^-1
3,BaselineOnly,3.367157,0.991174,2.691204,0.296986
0,SVD,3.507187,0.948571,10.297826,0.285129
1,SlopeOne,3.529565,0.895969,13.145518,0.283321
2,NMF,3.777866,0.854911,14.754872,0.264700
4,NormalPredictor,5.078288,0.697055,1.516193,0.196917


Based on the models above, it looks like the best performing model is BaselineOnly so i will be using this model. it is the simplest and the least costing collaborative filtering algorithms.

In [24]:
als_param_grid = {'bsl_options': {'method': ['als'],
                              'reg_i': [5, 10, 15],
                              'reg_u': [10, 15, 20],
                              'n_epochs': [5, 10, 15, 20]
                              }
              }

sgd_param_grid = {'bsl_options': {'method': ['sgd'],
                              'reg': [0.01, 0.02, 0.03],
                              'n_epochs': [5, 10, 15, 20],
                              'learning_rate' : [0.001, 0.005, 0.01]
                              }
              }

als_gs = sp.model_selection.GridSearchCV(sp.BaselineOnly, als_param_grid, measures=['rmse'], cv = 3, joblib_verbose = 0)

sgd_gs = sp.model_selection.GridSearchCV(sp.BaselineOnly, sgd_param_grid, measures=['rmse'], cv = 3, joblib_verbose = 0)

In [25]:
als_gs.fit(data)

# best RMSE score
print(als_gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(als_gs.best_params['rmse'])

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimati

In [26]:
sgd_gs.fit(data)

# best RMSE score
print(sgd_gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(sgd_gs.best_params['rmse'])

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimati

In [ ]:
trainset = data.build_full_trainset()
algo = sp.BaselineOnly()
algo.fit(trainset)
testset = trainset.build_anti_testset()
predictions = algo.test(testset)
    
last_predictions = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
last_predictions.drop('rui', inplace = True, axis = 1)

Estimating biases using als...
